Notes:
- Need to adjust city_bikes.ipynb to provide name and address of bike station (more valuable than lat,long as an output)
    - Then add processing for name and address of bike stations
- Need to either find a way to handle warnings, or to change all df.append to df.concat

## Set-up

In [1]:
# import libraries
from IPython.display import JSON #REMOVE ONCE NOTEBOOK IS COMPLETE
import requests
import pandas as pd
import os
import time

In [2]:
# set constants:
FOURSQUARE_API_KEY = os.getenv('FOURSQUARE_API_KEY')
YELP_API_KEY = os.getenv('YELP_API_KEY')
CALL_DELAY = 1 # pause (in s) after each call

In [3]:
# load data
palmaBikes = pd.read_csv('Statistical-Modelling/data/palmaBikes.csv')

# Foursquare

In [42]:
# define Foursquare specific functions

# format lat and long for Foursquare API
def getLatLong(cityBikes_df):
    ll_list = []
    for station in cityBikes_df.index:
        ll_list.append('{},{}'.format(str(cityBikes_df.iloc[station]['latitude']), str(cityBikes_df.iloc[station]['longitude'])))
    return ll_list


def getPOIs_FSQR(cityBikes_df):
    
    # initialize dataframe
    df = pd.DataFrame(columns = ['category', 'distance', 'fsq_id', 'name', 'rating', 'ratings'])
    
    # format latitude and longitude for Foursquare API
    cityBikes_df['ll'] = (pd.Series(getLatLong(palmaBikes)))
    
    for station in cityBikes_df.index:
        
        # make API call
        params = {
            'll': cityBikes_df.iloc[station]['ll'],
            'radius': '1000',
            'fields': ['categories', 'fsq_id', 'distance', 'name', 'rating', 'stats'],
            'sort': 'DISTANCE',
            'limit': '10'
        }
        
        url = f"https://api.foursquare.com/v3/places/search?ll={cityBikes_df.iloc[station]['ll']}&radius={params['radius']}&fields={params['fields'][0]}%2C{params['fields'][1]}%2C{params['fields'][2]}%2C{params['fields'][3]}%2C{params['fields'][4]}%2C{params['fields'][5]}&sort={params['sort']}&limit={params['limit']}"

        headers = {
            "Accept": "application/json",
            "Authorization": FOURSQUARE_API_KEY
        }

        response = requests.request("GET", url, headers=headers).json()
        time.sleep(CALL_DELAY)
        
        for POI in response['results']:
            # collecting category, distance, fsq_id, name, rating, ratings
            try:
                category = POI['categories'][0]['name']
            except:
                None
            distance = POI['distance']
            fsq_id = POI['fsq_id']
            name = POI['name']
            try:
                rating = POI['rating']
            except:
                rating = None
            try:
                ratings = POI['stats']['total_ratings']
            except:
                ratings = None

            # save to dataframe
            df = df.append({'category': category, 'distance': distance, 
                            'fsq_id': fsq_id, 'name': name, 'rating': rating,
                            'ratings':ratings}, ignore_index=True)
        
    return df

In [ ]:
results_FSQR = getPOIs_FSQR(palmaBikes)

In [44]:
results_FSQR.head()

,category,distance,fsq_id,name,rating,ratings
0,Farmers' Market,43,4bc84d218b7c9c748cd337cf,Mercat de Pere Garau,8.4,75
1,Café,108,4d331da3329e54811bbcb61d,Churreria la Artesana,6.2,8
2,Taco Restaurant,299,4f5e6a0ce4b0028cf960560b,Chapultepec,9.0,77
3,Grocery Store / Supermarket,313,4f6b5cc2e4b0b8a88349b5b2,Alimentos de China,None,None
4,Grocery Store / Supermarket,326,4cfe90b759c7b60c34f29f77,Mercadona,5.6,9
...,...,...,...,...,...,...
365,Tapas Restaurant,162,4bedb0d0767dc9b61152d3e9,Tast,7.8,92
366,Asian Restaurant,175,4baa0b0ef964a52022463ae3,Arume,8.3,70
367,Deli,175,4da32eb87ccc816e1773777b,Sa Formatgeria,8.0,14
368,Seafood Restaurant,183,4f4a410be4b04587825473be,Casa Gallega,7.5,210


# Yelp

In [54]:
#define yelp specific functions

def getPOIs_YELP(cityBikes_df): 

    # initialize dataframe
    df = pd.DataFrame(columns = ['API', 'yelp_id', 'category', 'distance', 'name', 'rating', 'ratings'])
    
    for station in cityBikes_df.index:
        
        # make API call
        params = {
            'latitude': cityBikes_df.iloc[0]['latitude'],
            'longitude': cityBikes_df.iloc[0]['longitude'],
            'radius': '1000',
            'limit': '10'
        }
        url = f"https://api.yelp.com/v3/businesses/search?latitude={params['latitude']}&longitude={params['longitude']}&radius={params['radius']}&sort_by=distance&limit={params['limit']}"
        headers = {
            "accept": "application/json",
            "Authorization": "bearer " + YELP_API_KEY
        }

        response = requests.get(url, headers=headers).json()
        time.sleep(CALL_DELAY)
                
        for POI in response['businesses']:
            
            # collect id, category, distance, name, rating, ratings
            yelp_id = POI['id']
            try:
                category = POI['categories'][0]['title']
            except:
                None
            distance = POI['distance']
            name = POI['name']
            rating = POI['rating']
            ratings = POI['review_count']
            
            # tagging as Yelp
            API = "YELP"
            
            # save to dataframe
            df = df.append({'API': API, 'yelp_id': yelp_id, 'category': category, 'distance': distance, 
                            'name': name, 'rating': rating,
                            'ratings':ratings}, ignore_index=True)
    return df

In [ ]:
results_YELP = getPOIs_YELP(palmaBikes)

In [56]:
results_YELP.head()

,API,yelp_id,category,distance,name,rating,ratings
0,YELP,BWWXOHCcfSNXl3yeGcErsw,Chinese,324.543739,Shi Shan Xuan,5.0,3
1,YELP,e3F0HeIfhCByxFN9hUP_cA,Bars,406.299598,Las Caribenas,5.0,1
2,YELP,8yYthxSx297ic_pIG5D3Lg,Tapas Bars,406.663624,Dehesa Santa Maria,4.0,1
3,YELP,4tGOSHJlKK2MXkwszcnbdQ,Bars,423.613087,Bar Monaco,4.0,1
4,YELP,JqjGid2vEcHBvxSbL2Oi5w,Spanish,444.985865,Palacio Chino,5.0,1
...,...,...,...,...,...,...,...
365,YELP,jgVbMhS5M_rAZwYAm-TW2A,Restaurants,464.481870,Cafe al Vent,4.0,1
366,YELP,xvXMSzFMiS_1ADKoJmMS5w,Latin American,465.835894,La Casa del Sabor,5.0,3
367,YELP,NFy_UNZZYJobnJRKhXHABQ,Indian,484.482969,Taj Mahal,5.0,1
368,YELP,CdnRufS1fW5s6193RV8PKw,Mediterranean,507.843243,Basic,5.0,8


# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating